This notebook plots the depth and area of various surveys that have been used to measure the UV Luminosity function.
I tried to list all of the current best-measurements: Hubble for deep, and HSC for wide.

LSST sources:
- DDFs: https://project.lsst.org/groups/sac/sites/lsst.org.groups.sac/files/Brandt_DDF.pdf
- Commissioning: https://project.lsst.org/meetings/rubin2022/sites/default/files/Rubin%20PCW%202022%20-%20Science%20Verification%20and%20Validation_0.pdf

Hubble sources:
- https://arxiv.org/abs/2102.07775
- https://arxiv.org/abs/1403.4295

JWST source:
- https://arxiv.org/abs/2110.13160

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
lsst_color = "C3"
stage3_color = "C0"
hubble_color = "k"
jwst_color = "C4"
euclid_color = "C2"

single = 0.15
single_under = -0.18
double = 0.25
double_under = -0.28

surveys = {
    # name: [area, depth, color, x_offset, y_offset]

    # LSST
    "LSST\n(wide comm)": [1_000, 26.4, lsst_color, 0, double],
    "LSST\n(deep comm)": [100, 27.7, lsst_color, 0, double],
    "LSST\n(wide)": [18_000, 27.3, lsst_color, 0, double],
    "LSST\n(deep)": [40, 28.9, lsst_color, 0, double],

    # HSC, numbers in GOLDRUSH IV
    "HSC\n(wide)": [288, 25.9, stage3_color, 0, double_under],
    "HSC\n(deep)": [17.6, 26.3, stage3_color, 0, double],
    "HSC\n(UD)": [2.6, 26.63, stage3_color, 0, double],

    # other DE Stage III experiments
    # nominal values. they don't have UV LFs published.
    "DES": [5000, 25.2, stage3_color, 0, single_under],
    "KiDS": [1500, 25.2, stage3_color, 0, single],

    # Hubble Fields
    "CANDELS\n(wide)": [0.19, 27.6, hubble_color, 0, double_under],
    "CANDELS\n(deep)": [0.03, 28.2, hubble_color, 0, double_under],
    "GOODS": [0.08, 28.5, hubble_color, 0, single],
    "HFF": [0.008, 29, hubble_color, 0, single_under],
    "HDF": [0.0018, 29, hubble_color, -6e-4, single_under],
    "HUDF\nParallel": [0.003, 29, hubble_color, 5e-4, double],
    "HUDF": [0.001, 30, hubble_color, 0, single],

    # JWST Fields
    "JADES\n(wide)": [0.0806, 29.25, jwst_color, 0, double],
    "JADES\n(deep)": [0.0128, 30, jwst_color, 0, double],
    "COSMOS-Webb": [0.6, 27.6, jwst_color, 0.7, single],
    "PRIMER": [0.105, 29, jwst_color, 0.35, 0],

    # Euclid
    "Euclid\n(wide)": [15_000, 25.3, euclid_color, 0, double],
    "Euclid\n(deep)": [40, 27.3, euclid_color, 0, double_under],
}

In [ ]:
def plot_surveys(lsst_comm=False, lsst_full=False, ir=False):
    fig, ax = plt.subplots(figsize=(4, 4), dpi=150)
    ax.set(
        xlabel="Area (deg$^2$)",
        ylabel="Depth (AB mag, $5\sigma$)",
        xscale="log",
        xlim=(3e-4, 8e4),
        ylim=(24.7, 30.5),
    )

    # plot the surveys
    for survey, (area, depth, color, x_offset, y_offset) in surveys.items():
        # skip LSST commissioning?
        if not lsst_comm and (survey == "LSST\n(wide comm)" or survey == "LSST\n(deep comm)"):
            continue
        # skip LSST full survey?
        if not lsst_full and (survey == "LSST\n(wide)" or survey == "LSST\n(deep)"):
            continue
        # skip JWST?
        if (color == jwst_color or color == euclid_color) and not ir:
            continue
        
        # plot the survey point
        ax.scatter(area, depth, marker="s", s=10, c=color)

        # set box around text
        if "LSST" in survey and not ("comm" in survey and lsst_full) and not ir:
            bbox = dict(facecolor="none", edgecolor="C3", pad=1, lw=0.5)
            y_offset += 0.07
        else:
            bbox = None

        # label the survey point
        ax.annotate(survey, (area+x_offset, depth+y_offset), fontsize=7, ha="center", va="center", color=color, bbox=bbox)

    # put the legend in the bottom left
    ax.text(0.01, 0.01, "Hubble", color=hubble_color, transform=ax.transAxes, va="bottom", ha="left", fontsize=9)
    ax.text(0.01, 0.06, "Ground", color=stage3_color, transform=ax.transAxes, va="bottom", ha="left", fontsize=9)
    if lsst_comm or lsst_full:
        ax.text(0.01, 0.11, "LSST", color=lsst_color, transform=ax.transAxes, va="bottom", ha="left", fontsize=9)
    if ir:
        ax.text(0.01, 0.16, "Euclid", color=euclid_color, transform=ax.transAxes, va="bottom", ha="left", fontsize=9)
        ax.text(0.01, 0.21, "JWST NIRCam", color=jwst_color, transform=ax.transAxes, va="bottom", ha="left", fontsize=9)

    # if we plotted the full LSST surveys, point from commissioning to science fields
    if lsst_comm and lsst_full and not ir:
        ax.annotate(
            "",
            xy=np.array(surveys["LSST\n(deep)"][:2]) + np.array([+3, -0.05]),
            xytext=np.array(surveys["LSST\n(deep comm)"][:2]) + np.array([-30, +0.45]),
            arrowprops=dict(arrowstyle="->", color="C3"),
        )
        ax.annotate(
            "",
            xy=np.array(surveys["LSST\n(wide)"][:2]) + np.array([-4000, -0.04]),
            xytext=np.array(surveys["LSST\n(wide comm)"][:2]) + np.array([1_000, 0.45]),
            arrowprops=dict(arrowstyle="->", color="C3"),
        )

    plt.tight_layout()

    return fig, ax

In [ ]:
fig, ax = plot_surveys(lsst_comm=False, lsst_full=False)
fig.savefig("figures/surveys_preLsst.png", dpi=500)

In [ ]:
fig, ax = plot_surveys(lsst_comm=True, lsst_full=False)
fig.savefig("figures/surveys_commLsst.png", dpi=500)

In [ ]:
fig, ax = plot_surveys(lsst_comm=True, lsst_full=True)
fig.savefig("figures/surveys_postLsst.png", dpi=500)

In [ ]:
fig, ax = plot_surveys(lsst_comm=True, lsst_full=True, ir=True)
fig.savefig("figures/surveys_withIR.png", dpi=500)